# Code to randomly split all usable images into test train and validate datasets

In [1]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import cv2
import pandas as pd
import pickle
import random

In [2]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [3]:
#load list of all images
all_list = pickle.load( open( "/home/ubuntu/efs/acrin_by_machine/machine_photos/list_save.pkl", "rb" ) )
#load the dataframe that determines if each photo is usable
df = pickle.load( open( "/home/ubuntu/efs/acrin_by_machine/machine_photos/df_save_new.pkl", "rb" ) )


In [5]:
#add a colummn to the dataframe that identifies the machine type of the photo
for index,row in df.iterrows():
    df.loc[index,'machine'] = row.fileName.split('/')[-2][-1]

In [10]:
pickle.dump( df, open( "/home/ubuntu/efs/acrin_by_machine/machine_photos/machine_df_full.pkl", "wb" ) )

In [11]:
#split all usable photos into train/validate/test and store each into a seperate directory
import shutil
import os
n_train = 0.8
n_valid = 0.1
n_test = 0.1
machine_list = list(set(df.machine))
for mach in machine_list:
    this_list = list(df[(df.machine==mach) & (df.good==1)].fileName)
    random.shuffle(this_list)
    directory = 'machine_'+mach
    num_train = int(len(this_list)*n_train)
    num_valid = int(len(this_list)*n_valid)
    num_test = int(len(this_list)*n_test)
    if not os.path.exists('/home/ubuntu/Part1/train/'+directory):
        os.makedirs('/home/ubuntu/Part1/train/'+directory)
    if not os.path.exists('/home/ubuntu/Part1/valid/'+directory):
        os.makedirs('/home/ubuntu/Part1/valid/'+directory)
    if not os.path.exists('/home/ubuntu/Part1/test/'+directory):
        os.makedirs('/home/ubuntu/Part1/test/'+directory)


    for i in range(0,num_train,1):
        shutil.copy(this_list[i], '/home/ubuntu/Part1/train/'+directory)
    for i in range(num_train,num_valid+num_train,1):
        shutil.copy(this_list[i], '/home/ubuntu/Part1/valid/'+directory)
    for i in range(num_train+num_valid,len(this_list),1):
        shutil.copy(this_list[i], '/home/ubuntu/Part1/test/'+directory)